In [5]:
from torch_geometric.data import HeteroData
import torch

In [6]:
def create_fake_datasets(n) -> HeteroData:
    datas = []
    for _ in range(n):
        data = HeteroData()
        data['activity'].x = torch.rand(size=[3,10]) # 3 activities X 10 features
        data['resource_static'].x = torch.rand(size=[1,5]) # 1 static resources X 5 features
        data['resource_dynamic'].x = torch.rand(size=[3,8]) # 1 dynamic resources(one node for each acivity so 3 in total) X 8 features
        data['attribute'].x = torch.rand(size=[3,2]) # 3 attributes(one for each activity) X 2 features
        data[('activity', 'follows', 'activity')].edge_attribute = torch.rand(size=[3,1]) # 3 edges a1a2 a2a3 a1a3 X 1 timedelta
        data[('activity', 'has', 'resource_static')].edge_attribute = torch.rand(size=[3,4]) # 3 edges one per activity X 4 features
        #data[('activity', 'has', 'resource_dynamic')].edge_attribute -> only connection
        data[('resource_dynamic', 'resource_delta', 'resource_dynamic')].edge_attribute = torch.rand(size=[2, 1]) # (n-1) activities X 1 delta resource
        data[('activity', 'has', 'attribute')].edge_index = torch.tensor([[0,1,2],[0,1,2]], dtype=torch.int64)
        data[('activity', 'follows', 'activity')].edge_index = torch.tensor([[0,1],[1,2]], dtype=torch.int64)
        data[('activity', 'has', 'resource_static')].edge_index = torch.tensor([[0,1,2],[0,0,0]], dtype=torch.int64)
        data[('resource_dynamic', 'resource_delta', 'resource_dynamic')].edge_index = torch.tensor([[0,1],[1,2]], dtype=torch.int64)
        data[('activity', 'has', 'resource_dynamic')].edge_index = torch.tensor([[0,1,2],[0,1,2]], dtype=torch.int64)
        datas.append(data)
    return datas

In [7]:
data = create_fake_datasets(1)[0]

In [8]:
from models.models import HGNN
model = HGNN(64, 10, 3, node_types=data.node_types)

In [9]:
data

HeteroData(
  activity={ x=[3, 10] },
  resource_static={ x=[1, 5] },
  resource_dynamic={ x=[3, 8] },
  attribute={ x=[3, 2] },
  (activity, follows, activity)={
    edge_attribute=[3, 1],
    edge_index=[2, 2],
  },
  (activity, has, resource_static)={
    edge_attribute=[3, 4],
    edge_index=[2, 3],
  },
  (resource_dynamic, resource_delta, resource_dynamic)={
    edge_attribute=[2, 1],
    edge_index=[2, 2],
  },
  (activity, has, resource_dynamic)={ edge_index=[2, 3] }
)

In [10]:
from torch_geometric.transforms import ToUndirected, NormalizeFeatures

print(ToUndirected()(data))


HeteroData(
  activity={ x=[3, 10] },
  resource_static={ x=[1, 5] },
  resource_dynamic={ x=[3, 8] },
  attribute={ x=[3, 2] },
  (activity, follows, activity)={
    edge_attribute=[3, 1],
    edge_index=[2, 4],
  },
  (activity, has, resource_static)={
    edge_attribute=[3, 4],
    edge_index=[2, 3],
  },
  (resource_dynamic, resource_delta, resource_dynamic)={
    edge_attribute=[4, 1],
    edge_index=[2, 4],
  },
  (activity, has, resource_dynamic)={ edge_index=[2, 3] },
  (resource_static, rev_has, activity)={
    edge_index=[2, 3],
    edge_attribute=[3, 4],
  },
  (resource_dynamic, rev_has, activity)={ edge_index=[2, 3] }
)


In [11]:
model

HGNN(
  (convs): ModuleList(
    (0-2): 3 x HeteroConv(num_relations=4)
  )
  (linear_nodes): Linear(64, 64, bias=True)
  (fc): Sequential(
    (0): Linear(256, 10, bias=True)
    (1): Softmax(dim=None)
  )
)

In [12]:
data.x_dict

{'activity': tensor([[0.0681, 0.2809, 0.0812, 0.8757, 0.3037, 0.4820, 0.2088, 0.2962, 0.5296,
          0.1189],
         [0.3683, 0.7927, 0.7913, 0.9112, 0.4847, 0.5423, 0.0658, 0.7105, 0.2218,
          0.9659],
         [0.0439, 0.4591, 0.8484, 0.8946, 0.8384, 0.0454, 0.4898, 0.4737, 0.2331,
          0.1536]]),
 'resource_static': tensor([[0.5605, 0.8834, 0.5161, 0.1392, 0.1174]]),
 'resource_dynamic': tensor([[0.0769, 0.5322, 0.1353, 0.1763, 0.1420, 0.1166, 0.5091, 0.9134],
         [0.6493, 0.4551, 0.1254, 0.1201, 0.0097, 0.3752, 0.3305, 0.6584],
         [0.4934, 0.0396, 0.6812, 0.9473, 0.4412, 0.7376, 0.8842, 0.1361]]),
 'attribute': tensor([[0.5049, 0.0161],
         [0.6966, 0.5027],
         [0.4213, 0.2934]])}

In [13]:
data.edge_index_dict

{('activity',
  'follows',
  'activity'): tensor([[0, 1],
         [1, 2]]),
 ('activity',
  'has',
  'resource_static'): tensor([[0, 1, 2],
         [0, 0, 0]]),
 ('resource_dynamic',
  'resource_delta',
  'resource_dynamic'): tensor([[0, 1],
         [1, 2]]),
 ('activity',
  'has',
  'resource_dynamic'): tensor([[0, 1, 2],
         [0, 1, 2]])}

In [14]:
with torch.no_grad():
    a = model(data.x_dict, data.edge_index_dict)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x192 and 256x10)

In [ ]:
a

tensor([0.0889, 0.0927, 0.1014, 0.0978, 0.0927, 0.1030, 0.1029, 0.1131, 0.1095,
        0.0981])

In [ ]:
len(a)

10

In [ ]:
b = a[1]

In [ ]:
b


tensor(0.0927)

In [ ]:
torch.flatten(torch.stack(b, dim=0))

TypeError: stack(): argument 'tensors' (position 1) must be tuple of Tensors, not Tensor